In [241]:
import numpy as np
import matplotlib.pyplot as plt

In [242]:
def linear_interpolation(x0, y0, x1, y1, x):
    """
    Perform linear interpolation to find the y value corresponding to x.

    Parameters:
    x0, y0: coordinates of the first point
    x1, y1: coordinates of the second point
    x: the x value to interpolate

    Returns:
    y: the interpolated y value
    """
    y = y0 + (y1 - y0) * (x - x0) / (x1 - x0)
    return y

# Example usage:
# x0, y0 = 0, 0
# x1, y1 = 10, 10
# x = 5
# y = linear_interpolation(x0, y0, x1, y1, x)
# print(f"The interpolated value at x={x} is y={y}")

## Given Constants

In [243]:

n_p = 1200

n_g_guess = 387

N_p = 19 # between 17 and 20
N_g = 0

P_d = 12

P = 5 # Power in HP
life_time = 15000 # in hours?


A_v = 10



## Calculations

In [244]:
#shock factor
k_o = 1.5

In [245]:
P_des = P * k_o
P_des

7.5

### Velocity ratio, output speed

In [246]:
##Velocity ratio

VR_guess = n_p / n_g_guess


In [247]:
##Number of teeth on gear

N_g = round(N_p * VR_guess)
N_g


59

In [248]:
VR = N_g/N_p
n_g = n_p / VR

VR, n_g

(3.1052631578947367, 386.4406779661017)

### Diameters, Center distances, pitch line speed

In [249]:
##Diameters

D_p = N_p / P_d
D_g = N_g / P_d

In [250]:
##Center distance

C = (N_p + N_g) / (2 * P_d)
C

3.25

In [251]:
##v_t

v_t = np.pi * D_p * n_p / 12
v_t

497.41883681838385

## Spur gear stuff follows

### Calculate J, I

In [252]:
#find J_p and J_G

J_p = 0.34
J_G = 0.41

I = 0.1

assert(I > 0); assert(J_p > 0); assert(J_G > 0)

### Calculate forces

In [253]:
W_t = 33000 * P / v_t
W_t

331.71240770731873

### Face width

In [254]:
#face width

F = 12 / P_d #nominal value

assert(F < 2.0 * D_p)
assert(F > 0.5 * D_p)
F

1.0

## choose a material - this part is sketch

seems to apply the same for all gears??

this part consists of finding numbers in tables

In [255]:
C_p = 2300 # for steel

In [256]:
# usually given - use table 9-5 in mott 
# A_v = 10
assert(A_v is not None)


In [257]:
k_v = 1.25 #from A_v

assert(k_v > 0)

In [258]:
#C_pf
C_pf = None

if F / D_p < 0.5:
    assert(False)
    #use graph
elif F <= 1.0:
    C_pf = F / (10 * D_p) - 0.025
elif F <= 15:
    C_pf = F / (10 * D_p) - 0.0375 + 0.0125 * F

assert(C_pf is not None)

C_pf


0.038157894736842106

In [259]:
#C_ma
cond = "commercial Enclosed".upper()
assert(cond is not None)

C_ma = -1

if cond == "commercial enclosed".upper():
    C_ma = 0.127 + 0.0158 * F - 1.093 * 10**-4 * F**2
elif cond == "precision".upper():
    C_ma = 0.0675 + 0.0128 * F - 0.926 * 10**-4 * F**2
elif cond == "extra".upper():
    C_ma = 0.0380 + 0.0102 * F - 0.822 * 10**-4 * F**2

C_ma


0.1426907

In [260]:
k_m = 1 + C_ma + C_pf
k_m

1.1808485947368421

### Find $k_s, k_B, k_R, k_v, SF$ from a table idrk

In [261]:
k_s = 1.0

In [262]:
#k_B
# assert(False)
#idk how to determine this
k_B = 1.00
k_B

1.0

In [263]:
k_R = 1.00

In [264]:
safety_factor = 1.0

## Calculate $Y_{np}, Y_{ng}$ with the above numbers

### Number of loading cycles

In [265]:
#Number of cycles

N_cp = 60 * life_time * n_p
N_cg = 60 * life_time * n_g

print(f"N_cp: {N_cp:.2e}, N_cg: {N_cg:.2e}")

N_cp: 1.08e+09, N_cg: 3.48e+08


### Calc $Y_{xx}, Z_{xx}$

In [266]:
# assert(False)
Y_np = 0.95
Y_ng = 1.0

Z_np = 0.898
Z_ng = 0.922

## Calculate $S_{tP}$ and $S_{tG}$

In [267]:
assert(F != 0)
assert(J_p != 0)

s_tp = (W_t * P_d) / (F * J_p) * k_o * k_s * k_m * k_B * k_v
s_tp

# W_t, P_d, F, J_p, k_o, k_s, k_m, k_B, k_v

25921.464518247463

In [268]:
s_tG = s_tp * J_p / J_G
s_tG

21495.848624888145

In [269]:
#idk what this is
s_atP_Greater_than = s_tp * k_R * safety_factor / Y_np
s_atP_Greater_than

27285.752124471015

In [270]:
#idk what this is
s_atG_Greater_than = s_tG * k_R * safety_factor / Y_ng
s_atG_Greater_than

21495.848624888145

### Calculate $S_c$

In [271]:
S_c = C_p * np.sqrt( (W_t * k_o * k_s * k_m  * k_v) / (F * D_p * I) ) #
S_c

156646.3433519493

### Find adjusted values of $S_c$

In [272]:
S_acP_Greater_than = S_c * (k_R * safety_factor) / Z_np
S_acP_Greater_than

174439.13513580099

In [273]:
S_acg_Greater_than = S_c * (k_R * safety_factor) / Z_ng
S_acg_Greater_than

169898.42012142006

## calculate brinell and power from the last part of the bible??

### Calculate the required brinell hardness for the pinion
 

In [274]:
# Prompt the user for input
HB_grade = input("HB grade 1 or 2? ")

Contact_HB_p = 0
Bending_HB_p = 0

if int(HB_grade) == 1:
    Contact_HB_p = ( S_acP_Greater_than / 1000 - 29.1 ) / 0.322
    Bending_HB_p = ( s_atP_Greater_than / 1000 - 12.8 ) / 0.0773

    Contact_HB_g = ( S_acg_Greater_than / 1000 - 29.1 ) / 0.322
    Bending_HB_g = ( s_atG_Greater_than / 1000 - 12.8 ) / 0.0773

elif int(HB_grade) == 2:
    Contact_HB_p = ( S_acP_Greater_than / 1000 - 34.3 ) / 0.349
    Bending_HB_p = ( s_atP_Greater_than / 1000 - 16.4 ) / 0.102

    Contact_HB_g = ( S_acg_Greater_than / 1000 - 34.3 ) / 0.349
    Bending_HB_g = ( s_atG_Greater_than / 1000 - 16.4 ) / 0.102

else:
    assert(False)


print(f"Contact_HB_p: {Contact_HB_p:.2f} psi \nBending_HB_p: {Bending_HB_p:.2f} psi\n")

print(f"Contact_HB_g: {Contact_HB_g:.2f} psi \nBending_HB_g: {Bending_HB_g:.2f} psi")

Contact_HB_p: 451.36 psi 
Bending_HB_p: 187.40 psi

Contact_HB_g: 437.26 psi 
Bending_HB_g: 112.49 psi


Use tables in the bible to find an appropriate material that satisfies the required HB. The material should be the same for the gear and the pinion.

Make sure to use table 9-8 form Mott to determine the heat treatment

Get $S_{acp}, S_{atp}$ for the material

In [281]:
S_acp = 178000#None
S_atp = 48400#None

S_acg = 1#None
S_atg = 1#None

In [282]:
contact_SF_p = S_acp * Y_np / (S_c * k_R)
bending_SF_p = S_atp * Y_np / (s_tp * k_R)

print(f"Contact Safety Factor (Pinion): {contact_SF_p:f} \nBending Safety Factor (Pinion): {bending_SF_p:f}\n")

contact_SF_g = S_acg * Y_np / (S_c * k_R)
bending_SF_g = S_atg * Y_np / (s_tG * k_R)

print(f"Contact Safety Factor (Gear): {contact_SF_g:.2f} \nBending Safety Factor (Gear): {bending_SF_g:.2f}")

print(S_acp, Y_np, S_c, k_R)


Contact Safety Factor (Pinion): 1.079502 
Bending Safety Factor (Pinion): 1.773820

Contact Safety Factor (Gear): 0.00 
Bending Safety Factor (Gear): 0.00
178000 0.95 156646.3433519493 1.0


In [277]:
# Power transmitting capacity

P_cap = n_p * F * I / (126000 * k_o * k_s * k_m * k_v) * ( (S_acP_Greater_than * D_p * Z_np) / (safety_factor * k_R * C_p) ) ** 2

print(f"P_cap = {P_cap:.4f} HP")

P_cap = 5.0020 HP
